In [3]:
import gym
import numpy as np
import numpy.matlib as npmat
import matplotlib.pyplot as plt
from collections import deque
import tensorflow as tf
import random
import threading
#from Q_agente_modelo import Q_agente_DNN, Target_Q_agente_DNN, toggle_rendering

""" Parameters """
def toggle_rendering():
    """
    This is just a helper function to toggle the environment
    rendering or quit the simulation
    """
    global render
    global runflag
    while runflag:
        p = raw_input()
        if p == 'q':
            runflag = False
        else:
            render = not render
            
            
            
class Q_agente_DNN(object):   
    
    def __init__(self,
                 dim_input,      
                 dim_hidden1,    
                 dim_hidden2,    
                 n_accao,        
                 gamma,
                 epsilon,
                 learning_rate,  
                 regularisation_l=0.0,
                 momentum=0.2):
        
        # Sessão de serviço para correr o grafo
        self.sess = tf.Session()
        
        # Dimensões da rede
        self.n_input    = dim_input      #dimensão do input
        self.n_hidden_1 = dim_hidden1    #dimensão da primeira layer
        self.n_hidden_2 = dim_hidden2    #dimensão da segunda layer
        self.n_output   = n_accao        #número de acções, igual à dimensão do output da rede
       
        # Parâmetros de Aprendizagem, relacionados com o treino ou a função de custo
        self.learning_rate   = learning_rate                #learning rate do optimizador 
        self.gamma           = gamma                        #desconto de rewards futuras
        self.epsilon         = epsilon                      #probabilidade de escolher uma acção random
        
        # RIGGED
        #self.reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
        self.reg = 0.01  # Choose an appropriate one.
        
        # Variáveis de percepcção e actuação do agente, inclui observação, acção e reward correspondente
        # De notar que não precisamos da observação seguinte, visto estarmos a efectuar Q-learning em ambiente deterministico
        # acção será o argmax do Q_agente_target
        self.x = tf.placeholder("float", [None, self.n_input])  # Perfil do tensor de observação, contêm a dimensão do input
        self.r = tf.placeholder("float", [None, 1])             # Reward obtida
        self.q_target = tf.placeholder("float", [None, 1]) 
   
        self.a = tf.placeholder("float", [None, n_accao])             # Acção exercida pelo agente 
        self.future_rewards = tf.placeholder("float", [None])
        # No futuro usar tf.Variable_Scope para lidar de forma mais segura com tf.Variables
        # Dicionários com os pesos e biases da rede
        self.stddev = 0.1 # <== This greatly affects accuracy!! 
        self.weights = {
            'h1': tf.Variable(tf.random_normal([self.n_input, self.n_hidden_1], stddev=self.stddev)),
            'h2': tf.Variable(tf.random_normal([self.n_hidden_1, self.n_hidden_2], stddev=self.stddev)),
            'out': tf.Variable(tf.random_normal([self.n_hidden_2, self.n_output], stddev=self.stddev))
        }
        self.biases = {
            'b1': tf.Variable(tf.random_normal([self.n_hidden_1],stddev=self.stddev)),
            'b2': tf.Variable(tf.random_normal([self.n_hidden_2],stddev=self.stddev)),
            'out': tf.Variable(tf.random_normal([self.n_output],stddev=self.stddev))
        }
    
        # Create model
    def multilayer_perceptron(self,_X, _weights, _biases):
        layer_1 = tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])) 
        layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2']))
        #output  = tf.nn.relu(tf.matmul(layer_2, _weights['out']) + _biases['out'])
        output  = tf.matmul(layer_2, _weights['out']) + _biases['out']
        return output

    
    def construct_model_and_initialize(self):
        # Construct model

        
        self.pred = self.multilayer_perceptron(self.x, self.weights, self.biases)
        self.y  = tf.reduce_sum(tf.mul(self.pred,self.a), reduction_indices = 1)
        #weightL2loss = self.reg*tf.nn.l2_loss(self.weights['h1']) + self.reg*tf.nn.l2_loss(self.weights['h2']) + self.reg*tf.nn.l2_loss(self.weights['out'])
        #biasL2loss = self.reg*tf.nn.l2_loss(self.biases['b1']) + self.reg*tf.nn.l2_loss(self.biases['b1']) + self.reg*tf.nn.l2_loss(self.biases['out'])
        #self.future_rewards = tf.add(self.r,tf.mul(self.gamma,self.q_target))
        
        # Define loss and optimizer
        self.cost = tf.reduce_mean(tf.square(self.future_rewards - self.y)) #+ weightL2loss + biasL2loss
        self.optm = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost) # Adam Optimizer

        # Initializing the variables
        self.init = tf.initialize_all_variables()
        self.sess.run(self.init)
        print ("Network Ready And Initialized")
    
    def actua(self,obs):
        
        accao = np.argmax(self.sess.run(self.pred, feed_dict={self.x: obs}))
        
        return accao
    
   
    #def copiar_Q_agente(self):
    #    with tf.name_scope("target_network_update"):
    #        self.target_network_update = []
    #        for v_source, v_target in zip(self.q_network.variables(), self.target_q_network.variables()):
    #            # this is equivalent to target = (1-alpha) * target + alpha * source
    #            update_op = v_target.assign_sub(self.target_network_update_rate * (v_target - v_source))
    #            self.target_network_update.append(update_op)
    #        self.target_network_update = tf.group(*self.target_network_update)
        
    print("Agent Init")
    
class Target_Q_agente_DNN(Q_agente_DNN):
    
    def __init__(self, 
                 dim_input,      
                 dim_hidden1,    
                 dim_hidden2,    
                 n_accao,        
                 gamma,
                 epsilon,
                 learning_rate,  
                 regularisation_l=0.0,
                 momentum=0.2):
        super(Target_Q_agente_DNN, self).__init__(dim_input,      
                 dim_hidden1,    
                 dim_hidden2,    
                 n_accao,        
                 gamma,
                 epsilon,
                 learning_rate,  
                 regularisation_l,
                 momentum)
    # Definir aqui todas as operações de updates de variáveis. Initializar variáveis em operações recurrentes,
    # faz com que sejam recursivamente adicionadas novas variáveis com as mesmas funções. 
    def computacao_eficiente(self,q_network):
        
        self.op1=self.weights['h1'].assign(q_network.weights['h1'].eval(session = q_network.sess))
        self.op2=self.weights['h2'].assign(q_network.weights['h2'].eval(session = q_network.sess))
        self.op3=self.weights['out'].assign(q_network.weights['out'].eval(session = q_network.sess))
        
        self.op4=self.biases['b1'].assign(q_network.biases['b1'].eval(session = q_network.sess))
        self.op5=self.biases['b2'].assign(q_network.biases['b2'].eval(session = q_network.sess))
        self.op6=self.biases['out'].assign(q_network.biases['out'].eval(session = q_network.sess))
    
        self.sess.run(self.op1)
        self.sess.run(self.op2)
        self.sess.run(self.op3)
        self.sess.run(self.op4)
        self.sess.run(self.op5)
        self.sess.run(self.op6)
    #def copiar_pesos(self,q_network):
        
    #    self.sess.run(self.op1)
    #    self.sess.run(self.op2)
    #    self.sess.run(self.op3)
    #    self.sess.run(self.op4)
    #    self.sess.run(self.op5)
    #    self.sess.run(self.op6)
        #self.sess.run(self)
        #print q_network.weights['h1'].eval(session = q_network.sess)
            

            


nA = 3
episodes = 100  # number of episodes to run
time_lim = 20000  # max number of steps per episode
problem = 'MountainCar-v0'
gamma = 0.99  # discount factor
alpha = 0.0075  # learning rate
C = 1500  # target net update frequency
N = 500000  # maximum number of experiences stored in memory
REPLAY_MEMORY = 500000 #replay memory
avgRh = []  # to store average rewards for visualization
Qh = []  # to store value from the initial state for visualization
Vfh = []
Rw = deque([], 100)  # running history of rewards
render = False  # render the simulation
render_last = 2  # render the last N steps of the simulation
dim_h = 100  # width of each hidden layer  (best = 5)
batch_size = 8  # minibatch size  (best = 64)
epsilon = 0.5
eps_dec = 0.995
eps_min = 0.1
rdeq = deque([], 100)
max_costs_hist = []
min_costs_hist = []
med_costs_hist = []
H = npmat.zeros([7, N], dtype='float32')
hi = 0
hsize = 0




Agent Init


In [4]:
""" Simulation """
env = gym.make(problem)
Q=Q_agente_DNN(2,100,100,3,0.99,0.5,0.0075)
T=Target_Q_agente_DNN(2,100,100,3,0.99,0.5,0.0075)
Q.construct_model_and_initialize()
T.construct_model_and_initialize()
T.computacao_eficiente(Q)
#T.copiar_pesos(Q)
D = deque()
tr = threading.Thread(target=toggle_rendering)
runflag = True
tr.start()


try:
    for e in xrange(episodes):
        o = env.reset()  # returns initial observation
        oprev = np.array(o, dtype='float32')
        a = Q.actua([oprev])
        a_t = np.zeros([nA])
        a_t[0] = 1
        sumR = 0
        costs_hist = []
        hsize = 0
        hi = 0
        for t in xrange(time_lim):

            if not runflag:
                break

            #if render or episodes-e < render_last:
            #    env.render()

            (o, r, done, info) = env.step(a)
            onext = np.array(o, dtype='float32')
            if done:
                of = oprev
            rdeq.append(r)

            D.append((oprev,a_t,onext,r,done))
            
            if len(D) > REPLAY_MEMORY:
                D.popleft() 
            
            if (t+1) % C == 0:
                T.computacao_eficiente(Q)
                #T.copiar_pesos(Q)
            
            a_t = np.zeros([nA])

            
            """ Training the QNet """
            if len(D) >= batch_size:
                minibatch = random.sample(D, batch_size)

                s_j_batch = [d[0] for d in minibatch]
                a_batch = [d[1] for d in minibatch]
                s_j1_batch = [d[2] for d in minibatch]
                r_batch = [d[3] for d in minibatch]
                

                y_batch = []
                readout_j1_batch = T.pred.eval(session=T.sess,feed_dict = {T.x : s_j1_batch})
                for i in range(0, len(minibatch)):
                    # if terminal only equals reward
                    if minibatch[i][4]:
                        y_batch.append(r_batch[i])
                    else:
                        y_batch.append(r_batch[i] + gamma * np.max(readout_j1_batch[i]))
                
               
                Q.sess.run(Q.optm,feed_dict = {
                    Q.future_rewards : y_batch,
                    Q.a : a_batch,
                    Q.x : s_j_batch})
                

                if random.random() <= epsilon:
                    a = random.randrange(nA)
                    a_t[a] = 1
                else:
                    a = Q.actua([onext])
                    a_t[a] = 1
                    
            else:
                a = np.random.choice(nA)
                a_t[a] = 1
            sumR += r
            oprev = onext
            if done:
                break
        Rw.append(sumR)
        avgRh.append(np.mean(Rw))

        epsilon = epsilon*eps_dec if epsilon > eps_min else eps_min

        print "=== Episode %d complete ===" % e
        print "Avg r: ", np.mean(Rw)
        print "Instant sumR: ", sumR

        if not runflag:
            break

    print "=== Simulation Complete ==="
    print "Final score: %d" % avgRh[-1]

    """ Visualization of the results """
    f1 = plt.figure()
    spy = f1.add_subplot(111)
    spy.plot(avgRh, label='Avg. R over 100 eps.', linewidth=2.0)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.xlabel('Episodes')
    Q.sess.close()
    T.sess.close()
    
    plt.show()

except KeyboardInterrupt:
    runflag = False
    tr.join()


[2016-09-24 21:35:50,195] Making new env: MountainCar-v0


Network Ready And Initialized
Network Ready And Initialized
=== Episode 0 complete ===
Avg r:  -20000.0
Instant sumR:  -20000.0
=== Episode 1 complete ===
Avg r:  -12854.5
Instant sumR:  -5709.0
=== Episode 2 complete ===
Avg r:  -10564.0
Instant sumR:  -5983.0
=== Episode 3 complete ===
Avg r:  -9679.75
Instant sumR:  -7027.0
=== Episode 4 complete ===
Avg r:  -10210.2
Instant sumR:  -12332.0
=== Episode 5 complete ===
Avg r:  -8685.0
Instant sumR:  -1059.0
=== Episode 6 complete ===
Avg r:  -10301.4285714
Instant sumR:  -20000.0
=== Episode 7 complete ===
Avg r:  -11505.0
Instant sumR:  -19930.0
=== Episode 8 complete ===
Avg r:  -12448.8888889
Instant sumR:  -20000.0
=== Episode 9 complete ===
Avg r:  -12744.1
Instant sumR:  -15401.0
=== Episode 10 complete ===
Avg r:  -12160.6363636
Instant sumR:  -6326.0
=== Episode 11 complete ===
Avg r:  -12813.9166667
Instant sumR:  -20000.0
=== Episode 12 complete ===
Avg r:  -12965.4615385
Instant sumR:  -14784.0
=== Episode 13 complete ===
A

In [3]:
minibatch

[(array([-0.46852192, -0.00151133], dtype=float32),
  array([ 0.,  1.,  0.]),
  array([-0.47044444, -0.00192253], dtype=float32),
  -1.0,
  False),
 (array([ -4.66921657e-01,   3.34080920e-04], dtype=float32),
  array([ 0.,  1.,  0.]),
  array([ -4.67010587e-01,  -8.89523071e-05], dtype=float32),
  -1.0,
  False),
 (array([-0.47176394, -0.0013195 ], dtype=float32),
  array([ 0.,  1.,  0.]),
  array([-0.47347066, -0.00170669], dtype=float32),
  -1.0,
  False),
 (array([-0.47044444, -0.00192253], dtype=float32),
  array([ 0.,  0.,  1.]),
  array([-0.47176394, -0.0013195 ], dtype=float32),
  -1.0,
  False),
 (array([ -4.67255712e-01,  -2.45355535e-04], dtype=float32),
  array([ 0.,  0.,  1.]),
  array([ -4.66921657e-01,   3.34080920e-04], dtype=float32),
  -1.0,
  False),
 (array([-0.46818739,  0.0005907 ], dtype=float32),
  array([ 0.,  0.,  1.]),
  array([-0.46701038,  0.00117702], dtype=float32),
  -1.0,
  False),
 (array([-0.46701038,  0.00117702], dtype=float32),
  array([ 1.,  0.,  

In [40]:
y=Q.sess.run(Q.y,feed_dict = {
                Q.future_rewards : y_batch,
                Q.a : a_batch,
                Q.x : s_j_batch})
print y

[-0.17822835 -0.50298917 -0.50319928 -0.50297076 -0.50292581 -0.15190996
 -0.50289959 -0.15183537]


In [13]:
readout_j1_batch = T.pred.eval(session=T.sess,feed_dict = {T.x : s_j1_batch})
print readout_j1_batch

[[ 0.15540685  0.17972887 -0.04821161]
 [ 0.15538457  0.17974788 -0.04821653]
 [ 0.15554243  0.17982095 -0.04816839]
 [ 0.15536901  0.1797404  -0.04822126]
 [ 0.15533277  0.1797232  -0.04823238]
 [ 0.15539819  0.1797545  -0.04821236]
 [ 0.15531226  0.17971349 -0.04823864]
 [ 0.15554881  0.17982459 -0.04816639]]


In [15]:
for i in range(0, len(minibatch)):
                    # if terminal only equals reward
                    if minibatch[i][4]:
                        y_batch.append(r_batch[i])
                    else:
                        y_batch.append(r_batch[i] + gamma * np.max(readout_j1_batch[i]))

In [19]:
print y_batch

[-0.82206842303276062, -0.82204959928989407, -0.82197725474834438, -0.82205700486898425, -0.82207402884960179, -0.82204304933547978, -0.82208364725112915, -0.82197365522384647, -0.82206842303276062, -0.82204959928989407, -0.82197725474834438, -0.82205700486898425, -0.82207402884960179, -0.82204304933547978, -0.82208364725112915, -0.82197365522384647, -0.82206842303276062, -0.82204959928989407, -0.82197725474834438, -0.82205700486898425, -0.82207402884960179, -0.82204304933547978, -0.82208364725112915, -0.82197365522384647]


In [21]:
print pred

[[-0.15183972 -0.50319153 -0.17822835]
 [-0.15188438 -0.50298917 -0.17821157]
 [-0.15183708 -0.50319928 -0.17822964]
 [-0.15188892 -0.50297076 -0.17820966]
 [-0.15189964 -0.50292581 -0.1782054 ]
 [-0.15190996 -0.50305265 -0.17819068]
 [-0.15190576 -0.50289959 -0.17820299]
 [-0.15183537 -0.5032025  -0.17823055]]


In [56]:
a_batch

[array([ 0.,  0.,  1.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 1.,  0.,  0.])]

In [54]:
y

array([-0.17822835, -0.50298917, -0.50319928, -0.50297076, -0.50292581,
       -0.15190996, -0.50289959, -0.15183537], dtype=float32)

In [45]:
y= Q.sess.run(Q.pred, feed_dict={Q.x: s_j_batch})
y_t = T.sess.run(T.pred, feed_dict={T.x: s_j_batch})
print y
print y_t

[[-0.51449412 -0.2868346  -0.32623082]
 [-0.51469588 -0.28694007 -0.32635134]
 [-0.51538187 -0.28726935 -0.32675299]
 [-0.5151791  -0.28719196 -0.32663488]
 [-0.51399523 -0.28656989 -0.32593334]
 [-0.51437277 -0.28678417 -0.32615682]
 [-0.51565862 -0.28739598 -0.32691297]
 [-0.51381671 -0.28646675 -0.32582784]]
[[-0.51449412 -0.2868346  -0.32623082]
 [-0.51469588 -0.28694007 -0.32635134]
 [-0.51538187 -0.28726935 -0.32675299]
 [-0.5151791  -0.28719196 -0.32663488]
 [-0.51399523 -0.28656989 -0.32593334]
 [-0.51437277 -0.28678417 -0.32615682]
 [-0.51565862 -0.28739598 -0.32691297]
 [-0.51381671 -0.28646675 -0.32582784]]


In [31]:
y_batch = []
readout_j1_batch = T.pred.eval(session=T.sess,feed_dict = {T.x : s_j1_batch})
for i in range(0, len(minibatch)):
# if terminal only equals reward
    if minibatch[i][4]:
        y_batch.append(r_batch[i])
    else:
        y_batch.append(r_batch[i] + gamma * np.max(readout_j1_batch[i]))

In [44]:
T.computacao_eficiente(Q)

In [41]:
Q.sess.run(Q.weights['h1'])

array([[-0.11842299,  0.04439117,  0.16220994,  0.01512942,  0.06441902,
        -0.2058327 ,  0.15205504,  0.02889177,  0.05861564, -0.17261411,
         0.18193264, -0.15791897, -0.00820857,  0.17591423, -0.11611249,
        -0.01017192,  0.10892375,  0.08214349, -0.24542077,  0.06357118,
         0.11206867, -0.17252877, -0.13303795, -0.14100067, -0.02839502,
        -0.08539977,  0.11517923, -0.04498134,  0.03216136,  0.26617774,
         0.06585611,  0.05631948, -0.00068399,  0.11056302, -0.09624262,
        -0.08573872,  0.19438291,  0.02016669,  0.06718251, -0.09028285,
        -0.0637907 , -0.03176225, -0.05000495,  0.10110813,  0.07702964,
         0.17069994,  0.0562891 ,  0.09567304, -0.17517808,  0.10179514,
         0.05129729,  0.03390796,  0.0892757 ,  0.00649925, -0.01418992,
         0.0712105 ,  0.00742582, -0.17233565,  0.08699498, -0.16153537,
        -0.14114662,  0.04416197,  0.0282966 ,  0.03731935, -0.07315317,
        -0.06846406,  0.16159391,  0.00313542, -0.0

In [42]:
T.sess.run(T.weights['h1'])

array([[-0.09593325,  0.06577849,  0.16220994,  0.00877897,  0.06441902,
        -0.2058327 ,  0.15205504,  0.00950908,  0.05861564, -0.15026949,
         0.18193264, -0.16858712, -0.00905997,  0.17591423, -0.10095793,
        -0.01017192,  0.13051046,  0.08214349, -0.24693875,  0.08607934,
         0.11206867, -0.15008587, -0.11109424, -0.11856753, -0.04807801,
        -0.06388431,  0.11517923, -0.02398199,  0.01023276,  0.26617774,
         0.0456562 ,  0.07851242, -0.0133181 ,  0.11056302, -0.11705349,
        -0.08387447,  0.19438291,  0.02016669,  0.08960155, -0.09713677,
        -0.07537597, -0.03176225, -0.02784461,  0.10110813,  0.07702964,
         0.17069994,  0.0562891 ,  0.09567304, -0.15354276,  0.10179514,
         0.03488758,  0.05632363,  0.0892757 ,  0.02884839, -0.02712709,
         0.0712105 ,  0.00742582, -0.19226597,  0.10942382, -0.13944004,
        -0.11909824,  0.06528366,  0.0282966 ,  0.05918797, -0.05121944,
        -0.04766155,  0.16159391, -0.0051396 , -0.0